In [1]:
#### ::::: FirstEF benchmark ::::: ####

In [2]:
import re
import os
import sys
import math
import pybedtools
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq

In [3]:
dir_path = os.path.dirname(os.path.realpath('benchmark_FirstEF.ipynb'))

In [4]:
firstef_pred_dir = dir_path + "/../../data/promoter/FirstEF/HsFef_Promoter.txt"

In [5]:
firstef_pred_df = pd.read_csv(firstef_pred_dir, sep='\t',header=None, names=['chrom','start','end','prob','NA','strand'])
firstef_pred_df.drop(['NA'],inplace=True,axis=1)
firstef_pred_df['pred'] = [1 if prob > 0.5 else 0 for prob in firstef_pred_df['prob']]
firstef_pred_df = firstef_pred_df[~firstef_pred_df['chrom'].isin(['chr23','chr24'])]

In [6]:
# split positive and negative prediction, convert to bed
firstef_pred_pos_df = firstef_pred_df[firstef_pred_df['pred'] == 1]
firstef_pred_neg_df = firstef_pred_df[firstef_pred_df['pred'] == 0]
firstef_pred_pos_df.drop(['prob','pred'], inplace=True, axis=1)
firstef_pred_neg_df.drop(['prob','pred'], inplace=True, axis=1)
firstef_pred_pos_df['name'] = 'pos_' + firstef_pred_pos_df.index.astype('str')
firstef_pred_neg_df['name'] = 'neg_' + firstef_pred_neg_df.index.astype('str')
firstef_pred_pos_df['score'] = 1
firstef_pred_neg_df['score'] = 1
# reorder
firstef_pred_pos_df = firstef_pred_pos_df[['chrom','start','end','name','score','strand']]
firstef_pred_neg_df = firstef_pred_neg_df[['chrom','start','end','name','score','strand']]

/home/yjq6148/.local/lib/python3.6/site-packages/pandas/core/frame.py:3697: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)
/home/yjq6148/.conda/envs/deeplearning/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/home/yjq6148/.conda/envs/deeplearning/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/inde

In [7]:
firstef_pred_pos_df.head()

,chrom,start,end,name,score,strand
0,chr1,10696,11266,pos_0,1,+
1,chr1,24218,24788,pos_1,1,+
2,chr1,29260,29830,pos_2,1,+
3,chr1,28626,29196,pos_3,1,+
4,chr1,89691,90261,pos_4,1,+


In [8]:
firstef_pred_pos = pybedtools.BedTool.from_dataframe(firstef_pred_pos_df)
firstef_pred_neg = pybedtools.BedTool.from_dataframe(firstef_pred_neg_df)

In [9]:
# read in TATA and noTATA test file
tata_scan_df = pd.read_csv(dir_path + "/../../data/promoter/human_epdnew_hg38_TATA_scan_test.csv",index_col=0)
tata_scan_df['score'] = 1
notata_scan_df = pd.read_csv(dir_path + "/../../data/promoter/human_epdnew_hg38_noTATA_scan_test.csv",index_col=0)
notata_scan_df['score'] = 1

In [10]:
len(tata_scan_df)

54054

In [11]:
len(notata_scan_df)

491400

In [12]:
# convert to bedfile
tata_scan_bed = pybedtools.BedTool.from_dataframe(tata_scan_df.drop(['label','keys','seq','id'],axis=1)[['chrom','start','end','name','score','strand']])
notata_scan_bed = pybedtools.BedTool.from_dataframe(notata_scan_df.drop(['label','keys','seq','id'],axis=1)[['chrom','start','end','name','score','strand']])

In [45]:
# # read in TATA and noTATA bedfile
# tata_bed = pybedtools.BedTool(dir_path + "/../../data/promoter/human_epdnew_hg38_TATA.bed")
# notata_bed = pybedtools.BedTool(dir_path + "/../../data/promoter/human_epdnew_hg38_noTATA.bed")
# tata_prom_bed = tata_bed.slop(g=dir_path + "/../../data/genome/chrom.sizes",l=500,r=499) # -500 to 500
# notata_prom_bed = notata_bed.slop(g=dir_path + "/../../data/genome/chrom.sizes",l=500,r=499) # -500 to 500

In [12]:
# tata_prom_bed.head()

chr1	999596	1000596	HES4_1	1	-
 chr1	1012996	1013996	ISG15_1	1	+
 chr1	1918778	1919778	TMEM52_1	1	-
 chr1	2526095	2527095	PANK4_1	1	-
 chr1	2529762	2530762	HES5_1	1	-
 chr1	3530910	3531910	MEGF6_1	1	-
 chr1	3619117	3620117	TPRG1L_2	1	+
 chr1	3652015	3653015	TP73_1	1	+
 chr1	3690171	3691171	TP73_3	1	+
 chr1	6602368	6603368	KLHL21_1	1	-
 

In [46]:
# # first find all FirstEF predictions that overlaps with true prom -500 to 500
# # TATA
# tata_firstef_pred_pos = tata_prom_bed.intersect(firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
# #                                 f=0.9,
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )
# # noTATA
# notata_firstef_pred_pos = notata_prom_bed.intersect(firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
# #                                 f=0.9,
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )

In [42]:
# len(tata_firstef_pred_pos)

1395

In [44]:
# # then overlap scan peaks with firstef positive predictions
# # TATA
# tata_scan_bed_pos = tata_scan_bed.intersect(tata_firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )
# tata_scan_bed_neg = tata_scan_bed.intersect(tata_firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 v=True, # Only report those entries in A that have **no overlaps** with B.
#                                 )

# # noTATA
# notata_scan_bed_pos = notata_scan_bed.intersect(notata_firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )
# notata_scan_bed_neg = notata_scan_bed.intersect(notata_firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 v=True, # Only report those entries in A that have **no overlaps** with B.
#                                 )

In [65]:
# # then overlap scan peaks with firstef positive predictions
# # TATA
# tata_scan_bed_pos = tata_scan_bed.intersect(tata_firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )
# # tata_scan_bed_neg = tata_scan_bed.intersect(tata_firstef_pred_pos,
# #                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
# #                                 F=500E-9, # Minimum overlap required as a fraction of B
# #                                 v=True, # Only report those entries in A that have **no overlaps** with B.
# #                                 )
# tata_scan_bed_neg_1 = tata_scan_bed.intersect(firstef_pred_neg,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )

# tata_scan_bed_neg_2 = tata_scan_bed.intersect(firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 v=True, # Only report those entries in A that have **no overlaps** with B.
#                                 )
# tata_scan_bed_neg = tata_scan_bed_neg_1.cat(tata_scan_bed_neg_2, postmerge=False)

# # noTATA
# notata_scan_bed_pos = notata_scan_bed.intersect(notata_firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )
# # notata_scan_bed_neg = notata_scan_bed.intersect(notata_firstef_pred_pos,
# #                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
# #                                 F=500E-9, # Minimum overlap required as a fraction of B
# #                                 v=True, # Only report those entries in A that have **no overlaps** with B.
# #                                 )
# notata_scan_bed_neg_1 = notata_scan_bed.intersect(firstef_pred_neg,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )

# notata_scan_bed_neg_2 = notata_scan_bed.intersect(firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 v=True, # Only report those entries in A that have **no overlaps** with B.
#                                 )
# notata_scan_bed_neg = notata_scan_bed_neg_1.cat(notata_scan_bed_neg_2, postmerge=False)

In [21]:
# # align and find overlap peaks in prom_bed
# # TATA
# tata_scan_bed_pos = tata_scan_bed.intersect(firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )

# tata_scan_bed_neg_1 = tata_scan_bed.intersect(firstef_pred_neg,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )
# tata_scan_bed_neg_2 = tata_scan_bed.intersect(firstef_pred_neg,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 v=True, # Only report those entries in A that have **no overlaps** with B.
#                                 ).intersect(firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 v=True, # Only report those entries in A that have **no overlaps** with B.
#                                 )
# tata_scan_bed_neg = tata_scan_bed_neg_1.cat(tata_scan_bed_neg_2, postmerge=False)
# # tata_scan_bed_neg = tata_scan_bed_neg_1

# # noTATA
# notata_scan_bed_pos = notata_scan_bed.intersect(firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )

# notata_scan_bed_neg_1 = notata_scan_bed.intersect(firstef_pred_neg,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )

# notata_scan_bed_neg_2 = notata_scan_bed.intersect(firstef_pred_neg,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 v=True, # Only report those entries in A that have **no overlaps** with B.
#                                 ).intersect(firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 v=True, # Only report those entries in A that have **no overlaps** with B.
#                                 )
# notata_scan_bed_neg = notata_scan_bed_neg_1.cat(notata_scan_bed_neg_2, postmerge=False)
# # notata_scan_bed_neg = notata_scan_bed_neg_1

In [29]:
# # align and find overlap peaks in prom_bed
# # TATA
# tata_scan_bed_pos = tata_scan_bed.intersect(firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )

# tata_scan_bed_neg_1 = tata_scan_bed.intersect(firstef_pred_neg,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )
# tata_scan_bed_neg_2 = tata_scan_bed.intersect(firstef_pred_neg,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 v=True, # Only report those entries in A that have **no overlaps** with B.
#                                 )
# tata_scan_bed_neg = tata_scan_bed_neg_1.cat(tata_scan_bed_neg_2, postmerge=False)

# # noTATA
# notata_scan_bed_pos = notata_scan_bed.intersect(firstef_pred_pos,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )

# notata_scan_bed_neg_1 = notata_scan_bed.intersect(firstef_pred_neg,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 u=True, # Write the original A entry **once** if **any** overlaps found in B
#                                 )

# notata_scan_bed_neg_2 = notata_scan_bed.intersect(firstef_pred_neg,
#                                 s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
#                                 F=500E-9, # Minimum overlap required as a fraction of B
#                                 v=True, # Only report those entries in A that have **no overlaps** with B.
#                                 )
# notata_scan_bed_neg = notata_scan_bed_neg_1.cat(notata_scan_bed_neg_2, postmerge=False)

In [89]:
# align and find overlap peaks in prom_bed
# TATA
tata_scan_bed_pos = tata_scan_bed.intersect(firstef_pred_pos,
                                s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
                                F=1.0, # Minimum overlap required as a fraction of B
                                u=True, # Write the original A entry **once** if **any** overlaps found in B
                                )

tata_scan_bed_neg = tata_scan_bed.intersect(firstef_pred_pos,
                                s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
                                F=1.0, # Minimum overlap required as a fraction of B
                                v=True, # Only report those entries in A that have **no overlaps** with B.
                                )

# noTATA
notata_scan_bed_pos = notata_scan_bed.intersect(firstef_pred_pos,
                                s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
                                F=1.0, # Minimum overlap required as a fraction of B
                                u=True, # Write the original A entry **once** if **any** overlaps found in B
                                )

notata_scan_bed_neg = notata_scan_bed.intersect(firstef_pred_pos,
                                s=True, # Require same strandedness.  That is, only report hits in that overlap A on the **same** strand.
                                F=1.0, # Minimum overlap required as a fraction of B
                                v=True, # Only report those entries in A that have **no overlaps** with B.
                                )

In [90]:
print(len(tata_scan_bed_pos), len(tata_scan_bed_neg),len(notata_scan_bed_pos),len(notata_scan_bed_neg))

2564 51490 28248 463152


In [91]:
# convert to df
tata_prom_pos_df = tata_scan_bed_pos.to_dataframe()
tata_prom_neg_df = tata_scan_bed_neg.to_dataframe()
notata_prom_pos_df = notata_scan_bed_pos.to_dataframe()
notata_prom_neg_df = notata_scan_bed_neg.to_dataframe()

# add pred
tata_prom_pos_df['pred'] = 1
tata_prom_neg_df['pred'] = 0
notata_prom_pos_df['pred'] = 1
notata_prom_neg_df['pred'] = 0

# concatenate
tata_prom_df = pd.concat([tata_prom_pos_df,tata_prom_neg_df])
notata_prom_df = pd.concat([notata_prom_pos_df, notata_prom_neg_df])

In [92]:
tata_prom_df.head()

,chrom,start,end,name,score,strand,pred
0,chr7,143315211,143316211,FP001328_CLCN1_1_2407,1,+,1
1,chr7,143315311,143316311,FP001328_CLCN1_1_2408,1,+,1
2,chr7,143315411,143316411,FP001328_CLCN1_1_2409,1,+,1
3,chr7,143315511,143316511,FP001328_CLCN1_1_2410,1,+,1
4,chr17,7854177,7855177,FP002383_NAA38_3_4017,1,+,1


In [93]:
# merge with reference
tata_prom_ref = pd.read_csv(dir_path + "/../../data/promoter/human_epdnew_hg38_TATA_scan_test.csv",index_col=0)
tata_prom_df = tata_prom_df.merge(tata_prom_ref, on='name')
notata_prom_ref = pd.read_csv(dir_path + "/../../data/promoter/human_epdnew_hg38_noTATA_scan_test.csv",index_col=0)
notata_prom_df = notata_prom_df.merge(notata_prom_ref, on='name')

In [94]:
## metrics
# TATA
TP = len(tata_prom_df[(tata_prom_df['label'] == 1) & (tata_prom_df['pred'] == 1)])
FP = len(tata_prom_df[(tata_prom_df['label'] == 0) & (tata_prom_df['pred'] == 1)])
FN = len(tata_prom_df[(tata_prom_df['label'] == 1) & (tata_prom_df['pred'] == 0)])
TN = len(tata_prom_df[(tata_prom_df['label'] == 0) & (tata_prom_df['pred'] == 0)])

accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP + FP)
recall = TP/(TP + FN)
F1 = 2*precision*recall/(precision+recall)
MCC = (TP*TN - FP*FN)/(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**0.5)

print("accuracy = ", accuracy)
print("precision = ", precision)
print("recall = ", recall)
print("F1 = ", F1)
print("MCC = ", MCC)

accuracy =  0.9078329078329078
precision =  0.3720748829953198
recall =  0.22052704576976423
F1 =  0.2769230769230769
MCC =  0.24017601957700835


In [95]:
# noTATA
TP = len(notata_prom_df[(notata_prom_df['label'] == 1) & (notata_prom_df['pred'] == 1)])
FP = len(notata_prom_df[(notata_prom_df['label'] == 0) & (notata_prom_df['pred'] == 1)])
FN = len(notata_prom_df[(notata_prom_df['label'] == 1) & (notata_prom_df['pred'] == 0)])
TN = len(notata_prom_df[(notata_prom_df['label'] == 0) & (notata_prom_df['pred'] == 0)])

accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP + FP)
recall = TP/(TP + FN)
F1 = 2*precision*recall/(precision+recall)
MCC = (TP*TN - FP*FN)/(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**0.5)

print("accuracy = ", accuracy)
print("precision = ", precision)
print("recall = ", recall)
print("F1 = ", F1)
print("MCC = ", MCC)

accuracy =  0.9054477004477004
precision =  0.4746530727839139
recall =  0.2977504385867514
F1 =  0.36594385840418125
MCC =  0.3278536996851556


In [96]:
# combined
scan_dev = pd.concat([tata_prom_df, notata_prom_df])

## metrics
TP = len(scan_dev[(scan_dev['label'] == 1) & (scan_dev['pred'] == 1)])
FP = len(scan_dev[(scan_dev['label'] == 0) & (scan_dev['pred'] == 1)])
FN = len(scan_dev[(scan_dev['label'] == 1) & (scan_dev['pred'] == 0)])
TN = len(scan_dev[(scan_dev['label'] == 0) & (scan_dev['pred'] == 0)])

accuracy = (TP+TN)/(TP+TN+FP+FN)
precision = TP/(TP + FP)
recall = TP/(TP + FN)
F1 = 2*precision*recall/(precision+recall)
MCC = (TP*TN - FP*FN)/(((TP+FP)*(TP+FN)*(TN+FP)*(TN+FN))**0.5)

print("accuracy = ", accuracy)
print("precision = ", precision)
print("recall = ", recall)
print("F1 = ", F1)
print("MCC = ", MCC)

accuracy =  0.905684072350739
precision =  0.46611709723484357
recall =  0.2909820288915453
F1 =  0.358293105813968
MCC =  0.32038125575954496
